In [ ]:
from google.colab import drive

# 1. Google Drive 마운트
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 2. 파일 경로 설정 (사용자 지정 경로)
data_root = '/content/drive/MyDrive/Colab Notebooks/datathon'
# file_path = f'{data_root}/2019-Nov.csv'

#데이터 불러오기 + 기본 전처리

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import timedelta

구매 빈도가 낮은 고객 분석 → 이탈 징후 탐지 및 리텐션 전략 수립
#"구매 빈도가 낮은 고객들의 이탈 징후를 파악하고, 재방문·재구매를 유도할 수 있는 전략 수립"

# GMP+5 적용 후 3월, 4월 분할

In [ ]:
df_3 = pd.read_parquet(f'{data_root}/2020-Mar.parquet')

# df_3.head()

In [ ]:
import pytz

# GMT+5 기준 datetime 객체 생성 (임시 변환)
event_time_gmt5 = df_3['event_time'].dt.tz_convert(pytz.FixedOffset(5 * 60))

# 기존 컬럼 수정 + 새 요일 컬럼 생성
df_3['event_date'] = event_time_gmt5.dt.date              # GMT+5 기준 날짜로 수정
df_3['event_hour'] = event_time_gmt5.dt.hour              # GMT+5 기준 시간으로 수정
df_3['weekday'] = event_time_gmt5.dt.day_name()           # GMT+5 기준 요일 새로 생성

In [ ]:
# df_3.tail()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,event_date,event_hour,category_lvl1,category_lvl2,category_lvl3,category_lvl4,weekday
56341236,2020-03-31 23:59:55+00:00,view,1005212,2232732093077520756,construction.tools.light,samsung,178.12,516506125,cdd4bce1-ff8c-49c3-92b3-0c7a209c8b24,2020-04-01,4,construction,tools,light,None,Wednesday
56341237,2020-03-31 23:59:55+00:00,view,1701027,2232732099981345295,appliances.personal.massager,samsung,154.42,572260610,2e16ffb5-996f-40d3-b98e-ddc53b546357,2020-04-01,4,appliances,personal,massager,None,Wednesday
56341238,2020-03-31 23:59:56+00:00,view,2601488,2232732101835227701,None,darina,167.29,523607693,6836d271-ef67-42dc-8756-4ef103e4fbf1,2020-04-01,4,None,None,None,None,Wednesday
56341239,2020-03-31 23:59:56+00:00,view,100006963,2232732099754852875,appliances.personal.massager,yasin,163.16,629229484,c00a1636-dfc2-4e96-a6a3-7fd979e93789,2020-04-01,4,appliances,personal,massager,None,Wednesday
56341240,2020-03-31 23:59:57+00:00,view,100145166,2232732076711346189,None,None,85.95,513238692,d33201ea-2f9d-4957-97fd-1efe9a315805,2020-04-01,4,None,None,None,None,Wednesday


In [ ]:
# 4/1 행만 필터링
df_4_1 = df_3[df_3['event_date'] == pd.to_datetime('2020-04-01').date()]

# # 4/1 데이터만 따로 저장
# output_path_4_1 = f'{data_root}/2020-04-01_data.parquet'
# df_4_1.to_parquet(output_path_4_1)

# print(f'Data for 2020-04-01 saved to: {output_path_4_1}')

Data for 2020-04-01 saved to: /content/drive/MyDrive/Colab Notebooks/datathon/2020-04-01_data.parquet


In [ ]:
# # df_3에서 4/1 행 제거
# df_3 = df_3[df_3['event_date'] != pd.to_datetime('2020-04-01').date()]

# # 수정된 df_3 다시 저장
# output_path = f'{data_root}/GMT_2020-Mar.parquet'
# df_3.to_parquet(output_path)

# print(f'Modified df_3 saved to: {output_path}')

Modified df_3 saved to: /content/drive/MyDrive/Colab Notebooks/datathon/GMT_2020-Mar.parquet


#4월 파일 통합

In [ ]:
# 1. 데이터 불러오기
df_4 = pd.read_parquet(f'{data_root}/2020-Apr.parquet')
df_4_1 = pd.read_parquet(f'{data_root}/2020-04-01_data.parquet')

# 2. 병합 (가급적 inplace처럼 변수 덮어쓰기)
df_4 = pd.concat([df_4, df_4_1], ignore_index=True)

# 3. 시간 기준 정렬
df_4 = df_4.sort_values(by='event_time').reset_index(drop=True)

# 4. 저장
df_4.to_parquet(f'{data_root}/GMT_2020-Apr.parquet')

print("✔ 병합 및 정렬 완료: GMT_2020-Apr.parquet")

In [ ]:
#세션 다운

#세션다운

In [ ]:
df_4 = pd.read_parquet(f'{data_root}/2020-Apr.parquet')

df_4.head()

NameError: name 'pd' is not defined

In [ ]:
df_4_combined = pd.concat([df_4_1, df_4], ignore_index=True)

# 시간 기준 정렬 (event_time 기준)
df_4_combined = df_4_combined.sort_values(by='event_time').reset_index(drop=True)

# 저장
df_4_combined.to_parquet(f'{data_root}/GMT_2020-Apr.parquet')

print("✔ 병합 및 정렬 완료: GMT_2020-Apr.parquet")

In [ ]:
#아악 ㅠ